In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from google.cloud import storage
import json
from scipy.io import loadmat
from pymatreader import read_mat
import os
import h5py
import gcsfs
import firebase_admin
from firebase_admin import db
import urllib.request

In [3]:
def firebase_connect():
    """Establishes connection  with Firebase in order to
       retreive and push data from and to Firebase realtime database and storage"""
    
    fs = gcsfs.GCSFileSystem(project='fit3162-cardio')
    with fs.open('firebase-cred/cardiomobilefyp-firebase-adminsdk-hekbv-8310eee5ad.json', 'rb') as cred_file:
        cred_dic = json.load(cred_file)
        cred_obj = firebase_admin.credentials.Certificate(cred_dic)
        default_app = firebase_admin.initialize_app(cred_obj, {
            'databaseURL': "https://cardiomobilefyp.firebaseio.com/"
            })
        
firebase_connect() #only needs to be run once after initalizing/restarting kernel

In [14]:
def newECGDataListener():
    
    """
     This function once called will always listen for changes in firebase realtime database for ECG_signal_data 
     Everytime it detects a change, it will fetch the changed data to here
     The data fetched will the downloadable url for the updated ecg signal file
     It will trigger under 2 conditions
         1. every 1 hour, does not care whether got any changes
         2. when there is a change 
    """
    listener_path = "ecg_queue"

    def listener(event):
        """This function is triggered when a new signal file upload from the mobile app is detected.
           Serves as an entry point to the cloud system for the mobile app."""

        # Trigger a https download procedure to download the file
        print(event.data)
        
        # extract out the filename from the downloadable url - To-do
        
        # downloads the file and store it under the current working directory
        print(len(event.data))
        
        if len(event.data) == 2:

            
            ecg_signal_data_url = event.data['ECG_signal_data']
            user_uid = event.data['uid']
            
            print(ecg_signal_data_url)
            print(user_uid)
            
            urllib.request.urlretrieve(ecg_signal_data_url, "noisy_ecg_signal_{}".format(user_uid))
            
            mobile_data_file = os.path.join(os.getcwd(), "noisy_ecg_signal_" + user_uid)
            signal_data = read_mat(mobile_data_file)
            rwb_fltrd_signal = rwb_filter(signal_data)
            signal_rpkeaks = signal_peaks(rwb_fltrd_signal)
            beats = slicer(rwb_fltrd_signal, signal_rpkeaks)
            heart_rate = bpm(signal_data[0], beats)
        
            padded_beats = trim_outliers(beats)
            print(padded_beats)
            pred_df = pd.DataFrame(padded_beats)
            #print(pred_df.head())
        
            input_data_norm = normalize(pred_df)
        
            predictions = get_predictions(input_data_norm)
        
            signal_class = classify_signal(predictions)
        
            uploadClassificationResult(signal_class, user_uid)
        
            uploadHeartRate(heart_rate, user_uid)
            
            
            db.reference("users/" + user_uid).update({"model_processing": "False"})


    """
     This listener function listens for child node ECG_signal_data in firebase 
     realtime database. If there is any changes to this node, it will 
     return the updated data here. Thus, every time the user uplaods
     their new ECG signal data, it detects and try to get the data for the model
     here to process
    """
    db.reference(listener_path).listen(listener)
    
newECGDataListener() #needs to be run once after initalizing/restarting kernel. Will constantly listen for new uploads until the notebook kernel is shutdown/restarted
    

{'-MaEL99Cx7yyXT2sa5MX': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00004_abnormaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=981f37ee-9f7f-46aa-84d0-ccc8eb0ca6b2', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELHA06whPgdR-ZCwb': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=8a4a8ffb-0783-47bf-a515-93ca5b9df219', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELhtHVQ-gRWUZMFBE': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=b05eb5d7-08c3-4df6-a7ce-a4b063e85e2c', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELlCYtkkhdB-RjgmQ': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.

In [17]:
def uploadClassificationResultCloud(model_result):
    """Used when the signal is uploaded to the storage bucket on Google Cloud.
    
       Function to push signal classification result to 
       the object at path below in Firebase realtime database"""
    
    path = "users/4BxcQ1NPHOUVGcICDK2EBffMzfv2/"
    db.reference(path).update({"classification_result" : model_result})

def uploadHeartRateCloud(ave_hr):
    """Used when the signal is uploaded to the storage bucket on Google Cloud.
    
       Function to push calculated signal heart rate to 
       the object at path below in Firebase realtime database"""
        
    path = "users/4BxcQ1NPHOUVGcICDK2EBffMzfv2/"
    db.reference(path).update({"heart_rate": ave_hr})    
    

    


def uploadClassificationResult(model_result, uid):
    """Used when the signal is uploaded from the mobile app.
    
       Function to push signal classification result to 
       the object at path below for user with ID = uid in Firebase realtime database"""
    
    user_path = "users"
    
    db.reference(user_path + "/" + uid).update({"classification_result" : model_result})

    
def uploadHeartRate(ave_hr, uid):
    """Used when the signal is uploaded from the mobile app.
    
       Function to push calculated signal heart rate to 
       the object at path below for user with ID = uid in Firebase realtime database"""

    user_path = "users"
    
    db.reference(user_path + "/" + uid).update({
        "heart_rate": ave_hr,
    })






def read_mat(mat_file):
    """Function to read signal files in .mat format and store the signal data in an array"""
    
    vals = []
    val_dict = loadmat(mat_file)
    #print(val_dict)
    vals.append([item for sublist in val_dict['val'] for item in sublist])
    #print(vals)
    return vals


def bpm(signal, beats):
    """Function to calculate heart rate for the uploaded signal.
    
        We know that the frequency for the provided ECG data is 300Hz (information provided with dataset)
        This would make the duration between each data timestamp(2 consecutive datapoints in the signal data) equal to 0.003 seconds.
        This information is used to calculate the heart rate for each signal after calculating the total length of the dignal and the
        number of beats in the signal"""
    
    print('Signal Length: ' + str(len(signal)))
    total_time = len(signal)*0.003
    total_beats = len(beats)
    #print(total_beats)
    rate_bpm  = (60*total_beats)/total_time
    print('BPM: ' + str(rate_bpm))
    return rate_bpm
     
    
def classify_signal(preds):
    """Function used to classify each incoming signal
    
        Threshold set at 60%. Any signal where 60% of the total beats are classified as normal by the
        model would be classified as normal, Otherwise, the signal would be classified as abnormal"""
    
    normal_beats = 0
    for i in range(len(preds)):
        if preds[i][0] == 1:
            normal_beats += 1
    if normal_beats/len(preds) > 0.60:
        print('Signal Classification: Normal')
        return 'normal'
    else:
        print('Signal Classification: Abnormal')
        return 'abnormal'
    

    
def load_signal():
    """Serves as an entry point for the entire signal processing and classification system. 
    
        Fetches a chosen .mat signal file from Cloud Storage Bucket and classifies it as normal or abnormal
        Requires Project ID and the cloud storage bucket path where the signal file is stored"""
    
    fs = gcsfs.GCSFileSystem(project='fit3162-cardio')

    with fs.open('saved-cardio-models/predict-data/signal.mat', 'rb') as model_file:
        signal_data = read_mat(model_file)
        rwb_fltrd_signal = rwb_filter(signal_data)
        signal_rpkeaks = signal_peaks(rwb_fltrd_signal)
        beats = slicer(rwb_fltrd_signal, signal_rpkeaks)
        heart_rate = bpm(signal_data[0], beats)
        
        padded_beats = trim_outliers(beats)
        print(padded_beats)
        pred_df = pd.DataFrame(padded_beats)
        #print(pred_df.head())
        
        input_data_norm = normalize(pred_df)
        
        predictions = get_predictions(input_data_norm)
        
        signal_class = classify_signal(predictions)
        
        uploadClassificationResultCloud(signal_class)
        
        uploadHeartRateCloud(heart_rate)
        
        db.reference("users/4BxcQ1NPHOUVGcICDK2EBffMzfv2/").update({"model_processing": "False"})
        

#uncomment and run this function (load_signal()) for testing signal processing and classification function 
#load_signal()


[ 127  342  560  797 1041 1272 1510 1754 1996 2230 2470 2713 2952 3190
 3433 3679 3915 4140 4371 4599 4827 5045 5260 5485 5711 5943 6165 6386
 6608 6827 7040 7259 7482 7698 7909 8128 8356 8586 8811]
Signal Length: 9000
BPM: 86.66666666666667
[[-23.52499325 -39.68481691 -56.60186896 ...   0.           0.
    0.        ]
 [265.84710461 240.75994611 178.91200422 ...   0.           0.
    0.        ]
 [240.26129045 202.54604829 141.53261713 ...   0.           0.
    0.        ]
 ...
 [346.48185154 264.40523094 154.7840506  ...   0.           0.
    0.        ]
 [290.08500435 204.55191257 100.93597521 ...   0.           0.
    0.        ]
 [388.30313305 325.81133373 220.32182349 ...   0.           0.
    0.        ]]
419
419
['saved-cardio-models/model/', 'saved-cardio-models/model/model.h5', 'saved-cardio-models/model/model1d.h5', 'saved-cardio-models/model/model1e.h5', 'saved-cardio-models/model/model1f.h5', 'saved-cardio-models/model/model2.h5']
X=[[0.         0.         0.         0.   

# Filtering ECG Signal

In [5]:
from scipy.signal import butter, filtfilt, iirnotch, savgol_filter



def butter_lowpass(cutoff, sample_rate, order=2):
    '''standard lowpass filter.

    Function that defines standard Butterworth lowpass filter

    Parameters
    ----------
    cutoff : int or float
        frequency in Hz that acts as cutoff for filter.
        All frequencies above cutoff are filtered out.

    sample_rate : int or float
        sample rate of the supplied signal

    order : int
        filter order, defines the strength of the roll-off
        around the cutoff frequency. Typically orders above 6
        are not used frequently.
        default: 2
    
    Returns
    -------
    out : tuple
        numerator and denominator (b, a) polynomials
        of the defined Butterworth IIR filter.

    Examples
    --------
    >>> b, a = butter_lowpass(cutoff = 2, sample_rate = 100, order = 2)
    >>> b, a = butter_lowpass(cutoff = 4.5, sample_rate = 12.5, order = 5)
    '''
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_highpass(cutoff, sample_rate, order=2):
    '''standard highpass filter.

    Function that defines standard Butterworth highpass filter

    Parameters
    ----------
    cutoff : int or float
        frequency in Hz that acts as cutoff for filter.
        All frequencies below cutoff are filtered out.

    sample_rate : int or float
        sample rate of the supplied signal

    order : int
        filter order, defines the strength of the roll-off
        around the cutoff frequency. Typically orders above 6
        are not used frequently.
        default : 2
    
    Returns
    -------
    out : tuple
        numerator and denominator (b, a) polynomials
        of the defined Butterworth IIR filter.

    Examples
    --------
    we can specify the cutoff and sample_rate as ints or floats.

    >>> b, a = butter_highpass(cutoff = 2, sample_rate = 100, order = 2)
    >>> b, a = butter_highpass(cutoff = 4.5, sample_rate = 12.5, order = 5)
    '''
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a


def butter_bandpass(lowcut, highcut, sample_rate, order=2):
    '''standard bandpass filter.
    Function that defines standard Butterworth bandpass filter.
    Filters out frequencies outside the frequency range
    defined by [lowcut, highcut].

    Parameters
    ----------
    lowcut : int or float
        Lower frequency bound of the filter in Hz

    highcut : int or float
        Upper frequency bound of the filter in Hz

    sample_rate : int or float
        sample rate of the supplied signal

    order : int
        filter order, defines the strength of the roll-off
        around the cutoff frequency. Typically orders above 6
        are not used frequently.
        default : 2
    
    Returns
    -------
    out : tuple
        numerator and denominator (b, a) polynomials
        of the defined Butterworth IIR filter.

    Examples
    --------
    we can specify lowcut, highcut and sample_rate as ints or floats.

    >>> b, a = butter_bandpass(lowcut = 1, highcut = 6, sample_rate = 100, order = 2)
    >>> b, a = butter_bandpass(lowcut = 0.4, highcut = 3.7, sample_rate = 72.6, order = 2)
    '''
    nyq = 0.5 * sample_rate
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def filter_signal(data, cutoff, sample_rate, order=2, filtertype='lowpass',
                  return_top = False):
    '''Apply the specified filter

    Function that applies the specified lowpass, highpass or bandpass filter to
    the provided dataset.

    Parameters
    ----------
    data : 1-dimensional numpy array or list 
        Sequence containing the to be filtered data

    cutoff : int, float or tuple
        the cutoff frequency of the filter. Expects float for low and high types
        and for bandpass filter expects list or array of format [lower_bound, higher_bound]

    sample_rate : int or float
        the sample rate with which the passed data sequence was sampled

    order : int
        the filter order 
        default : 2

    filtertype : str
        The type of filter to use. Available:
        - lowpass : a lowpass butterworth filter
        - highpass : a highpass butterworth filter
        - bandpass : a bandpass butterworth filter
        - notch : a notch filter around specified frequency range
        both the highpass and notch filter are useful for removing baseline wander. The notch
        filter is especially useful for removing baseling wander in ECG signals.


    Returns
    -------
    out : 1d array
        1d array containing the filtered data

    Examples
    --------
    >>> import numpy as np
    >>> import heartpy as hp

    Using standard data provided

    >>> data, _ = hp.load_exampledata(0)

    We can filter the signal, for example with a lowpass cutting out all frequencies
    of 5Hz and greater (with a sloping frequency cutoff)

    >>> filtered = filter_signal(data, cutoff = 5, sample_rate = 100.0, order = 3, filtertype='lowpass')
    >>> print(np.around(filtered[0:6], 3))
    [530.175 517.893 505.768 494.002 482.789 472.315]

    Or we can cut out all frequencies below 0.75Hz with a highpass filter:

    >>> filtered = filter_signal(data, cutoff = 0.75, sample_rate = 100.0, order = 3, filtertype='highpass')
    >>> print(np.around(filtered[0:6], 3))
    [-17.975 -28.271 -38.609 -48.992 -58.422 -67.902]

    Or specify a range (here: 0.75 - 3.5Hz), outside of which all frequencies
    are cut out.

    >>> filtered = filter_signal(data, cutoff = [0.75, 3.5], sample_rate = 100.0, 
    ... order = 3, filtertype='bandpass')
    >>> print(np.around(filtered[0:6], 3))
    [-12.012 -23.159 -34.261 -45.12  -55.541 -65.336]

    A 'Notch' filtertype is also available (see remove_baseline_wander).
    
    >>> filtered = filter_signal(data, cutoff = 0.05, sample_rate = 100.0, filtertype='notch')

    Finally we can use the return_top flag to only return the filter response that
    has amplitute above zero. We're only interested in the peaks, and sometimes
    this can improve peak prediction:

    >>> filtered = filter_signal(data, cutoff = [0.75, 3.5], sample_rate = 100.0, 
    ... order = 3, filtertype='bandpass', return_top = True)
    >>> print(np.around(filtered[48:53], 3))
    [ 0.     0.     0.409 17.088 35.673]
    '''
    if filtertype.lower() == 'lowpass':
        b, a = butter_lowpass(cutoff, sample_rate, order=order)
    elif filtertype.lower() == 'highpass':
        b, a = butter_highpass(cutoff, sample_rate, order=order)
    elif filtertype.lower() == 'bandpass':
        assert type(cutoff) == tuple or list or np.array, 'if bandpass filter is specified, \
cutoff needs to be array or tuple specifying lower and upper bound: [lower, upper].'
        b, a = butter_bandpass(cutoff[0], cutoff[1], sample_rate, order=order)
    elif filtertype.lower() == 'notch':
        b, a = iirnotch(cutoff, Q = 0.005, fs = sample_rate)
    else:
        raise ValueError('filtertype: %s is unknown, available are: \
lowpass, highpass, bandpass, and notch' %filtertype)

    filtered_data = filtfilt(b, a, data)
    
    if return_top:
        return np.clip(filtered_data, a_min = 0, a_max = None)
    else:
        return filtered_data



def remove_baseline_wander(data, sample_rate, cutoff=0.05):
    '''removes baseline wander

    Function that uses a Notch filter to remove baseline
    wander from (especially) ECG signals

    Parameters
    ----------
    data : 1-dimensional numpy array or list 
        Sequence containing the to be filtered data

    sample_rate : int or float
        the sample rate with which the passed data sequence was sampled

    cutoff : int, float 
        the cutoff frequency of the Notch filter. We recommend 0.05Hz.
        default : 0.05

    Returns
    -------
    out : 1d array
        1d array containing the filtered data

    Examples
    --------
    >>> import heartpy as hp
    >>> data, _ = hp.load_exampledata(0)

    baseline wander is removed by calling the function and specifying
    the data and sample rate.

    >>> filtered = remove_baseline_wander(data, 100.0)
    '''

    return filter_signal(data = data, cutoff = cutoff, sample_rate = sample_rate,
                         filtertype='notch')





def quotient_filter(RR_list, RR_list_mask = [], iterations=2):
    '''applies a quotient filter

    Function that applies a quotient filter as described in
    "Piskorki, J., Guzik, P. (2005), Filtering Poincare plots"

    Parameters
    ----------
    RR_list - 1d array or list
        array or list of peak-peak intervals to be filtered

    RR_list_mask - 1d array or list
        array or list containing the mask for which intervals are 
        rejected. If not supplied, it will be generated. Mask is 
        zero for accepted intervals, one for rejected intervals.

    iterations - int
        how many times to apply the quotient filter. Multipled
        iterations have a stronger filtering effect
        default : 2

    Returns
    -------
    RR_list_mask : 1d array
        mask for RR_list, 1 where intervals are rejected, 0 where
        intervals are accepted.

    Examples
    --------
    Given some example data let's generate an RR-list first
    >>> import heartpy as hp
    >>> data, timer = hp.load_exampledata(1)
    >>> sample_rate = hp.get_samplerate_mstimer(timer)
    >>> wd, m = hp.process(data, sample_rate)
    >>> rr = wd['RR_list']
    >>> rr_mask = wd['RR_masklist']

    Given this data we can use this function to further clean the data:
    >>> new_mask = quotient_filter(rr, rr_mask)

    Although specifying the mask is optional, as you may not always have a
    pre-computed mask available:
    >>> new_mask = quotient_filter(rr)
    
    '''

    if len(RR_list_mask) == 0:
        RR_list_mask = np.zeros((len(RR_list)))
    else:
        assert len(RR_list) == len(RR_list_mask), \
        'error: RR_list and RR_list_mask should be same length if RR_list_mask is specified'

    for iteration in range(iterations):
        for i in range(len(RR_list) - 1):
            if RR_list_mask[i] + RR_list_mask[i + 1] != 0:
                pass #skip if one of both intervals is already rejected
            elif 0.8 <= RR_list[i] / RR_list[i + 1] <= 1.2:
                pass #if R-R pair seems ok, do noting
            else: #update mask
                RR_list_mask[i] = 1
                #RR_list_mask[i + 1] = 1

    return np.asarray(RR_list_mask)


def smooth_signal(data, sample_rate, window_length=None, polyorder=3):
    '''smooths given signal using savitzky-golay filter

    Function that smooths data using savitzky-golay filter using default settings.

    Functionality requested by Eirik Svendsen. Added since 1.2.4

    Parameters
    ----------
    data : 1d array or list
        array or list containing the data to be filtered

    sample_rate : int or float
        the sample rate with which data is sampled

    window_length : int or None
        window length parameter for savitzky-golay filter, see Scipy.signal.savgol_filter docs.
        Must be odd, if an even int is given, one will be added to make it uneven.
        default : 0.1  * sample_rate

    polyorder : int
        the order of the polynomial fitted to the signal. See scipy.signal.savgol_filter docs.
        default : 3

    Returns
    -------
    smoothed : 1d array
        array containing the smoothed data

    Examples
    --------
    Given a fictional signal, a smoothed signal can be obtained by smooth_signal():

    >>> x = [1, 3, 4, 5, 6, 7, 5, 3, 1, 1]
    >>> smoothed = smooth_signal(x, sample_rate = 2, window_length=4, polyorder=2)
    >>> np.around(smoothed[0:4], 3)
    array([1.114, 2.743, 4.086, 5.   ])

    If you don't specify the window_length, it is computed to be 10% of the 
    sample rate (+1 if needed to make odd)
    >>> import heartpy as hp
    >>> data, timer = hp.load_exampledata(0)
    >>> smoothed = smooth_signal(data, sample_rate = 100)

    '''

    if window_length == None:
        window_length = sample_rate // 10
        
    if window_length % 2 == 0 or window_length == 0: window_length += 1

    smoothed = savgol_filter(data, window_length = window_length,
                             polyorder = polyorder)

    return smoothed

In [6]:
def rwb_filter(signals):
    """Function to filter incoming ECG signal data using the Notch filter
       in the scipy library"""
    
    fltr_signal = []
    for i in range(len(signals)):
        fltr_signal.append(remove_baseline_wander(signals[i], 300))
    return fltr_signal

# Pan-Tompkins QRS Algorithm

In [7]:
import math
from numpy import genfromtxt
import matplotlib.pyplot as plt

def read_ecg(file_name):
    return genfromtxt(file_name, delimiter=',')

def lgth_transform(ecg, ws):
    lgth=ecg.shape[0]
    sqr_diff=np.zeros(lgth)
    diff=np.zeros(lgth)
    ecg=np.pad(ecg, ws, 'edge')
    for i in range(lgth):
        temp=ecg[i:i+ws+ws+1]
        left=temp[ws]-temp[0]
        right=temp[ws]-temp[-1]
        diff[i]=min(left, right)
        diff[diff<0]=0
    # sqr_diff=np.multiply(diff, diff)
    # diff=ecg[:-1]-ecg[1:]
    # sqr_diff[:-1]=np.multiply(diff, diff)
    # sqr_diff[-1]=sqr_diff[-2]
    return np.multiply(diff, diff)

def integrate(ecg, ws):
    lgth=ecg.shape[0]
    integrate_ecg=np.zeros(lgth)
    ecg=np.pad(ecg, math.ceil(ws/2), mode='symmetric')
    for i in range(lgth):
        integrate_ecg[i]=np.sum(ecg[i:i+ws])/ws
    return integrate_ecg

def find_peak(data, ws):
    lgth=data.shape[0]
    true_peaks=list()
    for i in range(lgth-ws+1):
        temp=data[i:i+ws]
        if np.var(temp)<5:
            continue
        index=int((ws-1)/2)
        peak=True
        for j in range(index):
            if temp[index-j]<=temp[index-j-1] or temp[index+j]<=temp[index+j+1]:
                peak=False
                break

        if peak is True:
            true_peaks.append(int(i+(ws-1)/2))
    return np.asarray(true_peaks)

def find_R_peaks(ecg, peaks, ws):
    num_peak=peaks.shape[0]
    R_peaks=list()
    for index in range(num_peak):
        i=peaks[index]
        if i-2*ws>0 and i<ecg.shape[0]:
            temp_ecg=ecg[i-2*ws:i]
            R_peaks.append(int(np.argmax(temp_ecg)+i-2*ws))
    #print('R-peak function')
    #print(R_peaks)
    return np.asarray(R_peaks)

def find_S_point(ecg, R_peaks):
    num_peak=R_peaks.shape[0]
    S_point=list()
    for index in range(num_peak):
        i=R_peaks[index]
        cnt=i
        if cnt+1>=ecg.shape[0]:
            break
        while ecg[cnt]>ecg[cnt+1]:
            cnt+=1
            if cnt>=ecg.shape[0]:
                break
        S_point.append(cnt)
    return np.asarray(S_point)


def find_Q_point(ecg, R_peaks):
    num_peak=R_peaks.shape[0]
    Q_point=list()
    for index in range(num_peak):
        i=R_peaks[index]
        cnt=i
        if cnt-1<0:
            break
        while ecg[cnt]>ecg[cnt-1]:
            cnt-=1
            if cnt<0:
                break
        Q_point.append(cnt)
    return np.asarray(Q_point)

def EKG_QRS_detect(ecg, fs, QS, plot=False):
    sig_lgth=ecg.shape[0]
    ecg=ecg-np.mean(ecg)
    ecg_lgth_transform=lgth_transform(ecg, int(fs/20))
    # ecg_lgth_transform=lgth_transform(ecg_lgth_transform, int(fs/40))

    ws=int(fs/8)
    ecg_integrate=integrate(ecg_lgth_transform, ws)/ws
    ws=int(fs/6)
    ecg_integrate=integrate(ecg_integrate, ws)
    ws=int(fs/36)
    ecg_integrate=integrate(ecg_integrate, ws)
    ws=int(fs/72)
    ecg_integrate=integrate(ecg_integrate, ws)

    peaks=find_peak(ecg_integrate, int(fs/10))
    R_peaks=find_R_peaks(ecg, peaks, int(fs/40))
    if QS:
        S_point=find_S_point(ecg, R_peaks)
        Q_point=find_Q_point(ecg, R_peaks)
    else:
        S_point=None
        Q_point=None
    if plot:
        index=np.arange(sig_lgth)/fs
        fig, ax=plt.subplots()
        ax.plot(index, ecg, 'b', label='ECG')
        ax.plot(R_peaks/fs, ecg[R_peaks], 'ro', label='R peaks')
        if QS:
            ax.plot(S_point/fs, ecg[S_point], 'go', label='S')
            ax.plot(Q_point/fs, ecg[Q_point], 'yo', label='Q')
        ax.set_xlim([0, sig_lgth/fs])
        ax.set_xlabel('Time [sec]')
        ax.legend()
        # ax[1].plot(ecg_integrate)
        # ax[1].set_xlim([0, ecg_integrate.shape[0]])
        # ax[2].plot(ecg_lgth_transform)
        # ax[2].set_xlim([0, ecg_lgth_transform.shape[0]])
        #fig.figure(figsize=(20,10))
        plt.show()
    return R_peaks, S_point, Q_point

In [8]:
def QRS(data_list):
    '''
    QRS detection on 1-D numpy array
    300 Hz sampling rate
    '''
    fs=300
    R_peaks, S_point, Q_point=EKG_QRS_detect(np.asarray(data_list), fs, False)
    print(R_peaks)
    return R_peaks


def signal_peaks(signals):
    """Function for detecting R-peaks in the signal"""
    
    peaks = []
    for i in range(len(signals)):
        peaks.append(QRS(signals[i]))
    return peaks  

# Slicing signal into beats

In [9]:
def slice_signal(rpeaks, signal):
    """Function requires array with full signal and
       the array of the positions of detected R-peaks in the signal
       
       Outputs 2d array with identified beats in the signal using R-R peak segemntation"""
    
    signal_beats = []

    inx = 0
    if len(rpeaks) >= 10:
        for i in range(len(rpeaks)):
            #print('inx: ' + str(inx))
            #print('R-Peak Position: ' + str(rpeaks[i]))
            signal_beats.append(signal[inx:rpeaks[i]+1])
            #beat_lengths.append()
            inx = rpeaks[i]+1
        
    return signal_beats


def slicer(full_signal, signal_peaks):
    beats = []
    for i in range(len(full_signal)):
        beats_arr = slice_signal(signal_peaks[i], full_signal[i])
        for j in range(len(beats_arr)):
            if len(beats_arr[j]) > 0:
                beats.append(beats_arr[j])
    return beats

## Removing Anomalies in the Dataset and Padding Beats with Zeroes

In [10]:

def trim_outliers(beats):
    """Function to remove anomalies and pad the identified beats with zeroes to make them of equal length"""
    trimmed = []
    for i in range(len(beats)):
        """The range here is determined using statistical data related to the signals in the provided dataset
           Mean length of beats for provided dataset: 245.8
           Standard Deviation of beat length for provided dataset: 87.16
           
           Beats from the original dataset which the classifier was trained on were restricted to 
           length range between (Mean - 2*Standard Deviation) and (Mean + 2*Standard Deviation)
           
           Assuming the data is normally distributed, this would give us a good representative sample of the entire dataset at the same time removing any anomallies/extreme values
           which could negatively effect the model performance
           
           The Max Beat Length after removing anomalies in the dataset which was used to train and evaluate the model was 420 and the Min Beat Length was 71
           
           Using the same length range here for incoming signals that would be classified by the model.
           The range would need to be adjusted for a different dataset"""
        
        if len(beats[i]) > 71 and len(beats[i]) < 420:
            trimmed.append(beats[i])
            
    #padding with zeroes to make beats of equal length
    padded = np.asarray([np.pad(a, (0, 419 - len(a)), 'constant', constant_values=0) for a in trimmed])
    
    return padded

## Normalizing Data

In [11]:
def normalize(dataframe):
    """Function to normalize data between range 0-1"""
    
    dataset = dataframe.values
    X = dataset[:,0:419].astype(float)
    scaler = MinMaxScaler()
    X_norm = pd.DataFrame(scaler.fit_transform(X))
    X_norm_vals = X_norm.values
    #print(X[1])
    #print(X_norm_vals[1])
    print(len(X_norm_vals[1]))
    print(len(X[1]))
    
    """Reshaping data to make it compatible with the input requirements of a LSTM RNN"""
    X_adj_dim = np.reshape(X_norm_vals, (X_norm_vals.shape[0], 1, X_norm_vals.shape[1]))
    
    return X_adj_dim

In [12]:
"""final-df.csv contains the full provided dataset
   
   Dataset used here to train different types of models and tune hyperparameters for improved accuracy"""

dataframe = pd.read_csv('gs://cardio-data/dataset/final-df.csv')
dataset = dataframe.values
#print(dataset)
X = dataset[:,0:419].astype(float)
scaler = MinMaxScaler()
X_norm = pd.DataFrame(scaler.fit_transform(X))
X_norm_vals = X_norm.values
print(X[1])
print(X_norm_vals[1])
print(len(X_norm_vals[1]))
print(len(X[1]))
Y = (dataset[:,419]).astype(int)
print(Y)

[ 2.40261290e+02  2.02546048e+02  1.41532617e+02  7.87628712e+01
  2.35661511e+01 -2.55127146e+01 -7.16778893e+01 -1.13958668e+02
 -1.49188440e+02 -1.53662135e+02 -1.35669070e+02 -1.20695775e+02
 -1.06050927e+02 -9.41137208e+01 -8.33396418e+01 -7.42584793e+01
 -6.64644877e+01 -5.85982583e+01 -5.23306616e+01 -4.63704822e+01
 -4.04514079e+01 -3.53201313e+01 -3.27697131e+01 -3.06638474e+01
 -2.89084312e+01 -2.84361892e+01 -2.72036687e+01 -2.61669568e+01
 -2.52797193e+01 -2.34911296e+01 -2.27435986e+01 -2.30037122e+01
 -2.32719086e+01 -2.35601576e+01 -2.38813248e+01 -2.42497467e+01
 -2.46818710e+01 -2.41858212e+01 -2.27394191e+01 -2.23003640e+01
 -2.08266838e+01 -1.82525182e+01 -1.54740071e+01 -1.23669896e+01
 -9.80379798e+00 -8.68106442e+00 -6.92627271e+00 -6.48334812e+00
 -6.32132238e+00 -6.43294997e+00 -5.81204388e+00 -5.44202880e+00
 -5.30637092e+00 -5.39900948e+00 -5.72408599e+00 -8.31846985e+00
 -2.03762789e+01 -2.82798931e+01 -3.03601219e+01 -2.87322598e+01
 -2.73682592e+01 -2.41848

### Training and Evaluating LSTM RNN using Train-Test split

In [8]:
"""This Model produced the best accuracy on test set"""

X_train, X_test, y_train, y_test = train_test_split(X_norm_vals, Y, test_size=0.20, random_state=3)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

#print(len(X_train))
#print(X_train.shape)
#print(X_train.ndim)
#print(len(X_train[0]))

def lstm_rnn():
    model = Sequential()
    model.add(LSTM(250, return_sequences=True, input_shape=(1, 419)))
    model.add(Dropout(0.2))
    model.add(LSTM(250, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    #print(model.summary())
    model.fit(X_train, y_train, epochs=200, batch_size=10)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    model.save("model1d.h5")
    print("Accuracy: " + str((scores[1]*100)))


lstm_rnn()

Epoch 1/200
329/329 [==============================] - 4s 6ms/step - loss: 0.6919 - accuracy: 0.5184
Epoch 2/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6658 - accuracy: 0.5668
Epoch 3/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6468 - accuracy: 0.5753
Epoch 4/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6402 - accuracy: 0.5945
Epoch 5/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6398 - accuracy: 0.6133
Epoch 6/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6267 - accuracy: 0.6200
Epoch 7/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6296 - accuracy: 0.6057
Epoch 8/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6218 - accuracy: 0.6228
Epoch 9/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6207 - accuracy: 0.6298
Epoch 10/200
329/329 [==============================] - 2s 6ms/step - loss: 0.6162 - accura

### Training and Evaluating LSTM RNN using k-fold Cross Validation

In [9]:
"""This is the model that produced the best accuracy on test set"""

seed = 3
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
X_modded = np.reshape(X_norm_vals, (X_norm_vals.shape[0], 1, X_norm_vals.shape[1]))
model_num = 1
for train, test in kfold.split(X_modded, Y):
    model = Sequential()
    model.add(LSTM(250, return_sequences=True, input_shape=(1, 419)))
    model.add(Dropout(0.2))
    model.add(LSTM(250, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # Fit the model
    model.fit(X_modded[train], Y[train], epochs=200, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X_modded[test], Y[test], verbose=0)
    model.save("model" + str(model_num) + ".h5")
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    model_num += 1
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


accuracy: 80.54%
accuracy: 82.48%
accuracy: 72.99%
accuracy: 82.24%
accuracy: 80.29%
accuracy: 78.59%
accuracy: 79.32%
accuracy: 82.48%
accuracy: 77.37%
accuracy: 76.59%
79.29% (+/- 2.88%)


### Loading Saved LSTM RNN for predicting class of unlabelled ECG Signal Beats

In [15]:
from tensorflow.keras.models import load_model

def get_predictions(input_data):
    """Function that loads a saved model file from cloud storage bucket
       The saved model is used to predict incoming signal beats as normal/abnormal"""
    
    fs = gcsfs.GCSFileSystem(project='fit3162-cardio')
    print(fs.ls('saved-cardio-models/model')) 

    with fs.open('saved-cardio-models/model/model2.h5', 'rb') as model_file:
        model_gcs = h5py.File(model_file, 'r')
        saved_lstm_rnn = load_model(model_gcs, compile = True)
        Xnew = input_data
        ynew = (saved_lstm_rnn.predict(Xnew) > 0.5).astype(int)    
        # show the inputs and predicted outputs
        for i in range(len(Xnew)):
            print("X=%s, Predicted=%s" % (Xnew[i], ynew[i])) 
        return ynew

{'-MaEL99Cx7yyXT2sa5MX': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00004_abnormaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=981f37ee-9f7f-46aa-84d0-ccc8eb0ca6b2', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELHA06whPgdR-ZCwb': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=8a4a8ffb-0783-47bf-a515-93ca5b9df219', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELhtHVQ-gRWUZMFBE': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.mat?alt=media&token=b05eb5d7-08c3-4df6-a7ce-a4b063e85e2c', 'uid': 'uEiFa4ROFIhWPzbCf0z4eprAlBI2'}, '-MaELlCYtkkhdB-RjgmQ': {'ECG_signal_data': 'https://firebasestorage.googleapis.com/v0/b/cardiomobilefyp.appspot.com/o/ECG_signals%2FA00001_normaluEiFa4ROFIhWPzbCf0z4eprAlBI2.